In [1]:
%load_ext autoreload
%autoreload 2

import time

import numpy as np
import torch
from torch.nn import functional as F
from torch import optim as torch_opt

from flare import trainer
from flare.callbacks import Checkpoint

from models.mnist import ConvNet
from models import datasets

device = torch.device('cpu')
pin_memory = False
if torch.cuda.is_available():
    device = torch.device('cuda')
    pin_memory = True
    torch.backends.cudnn.benchmark = True

In [2]:
def _ceil(x):
    return int(np.round(x))

callno = 0
scoreboard = dict()
def make_target_fn(model_prefix, _device, model_class, trn_gen, val_gen, n_epochs, image_sz, n_classes, hyperparams):
    def target_fn(hyperparam_values):
        global callno
        global scoreboard

        # Ensuring that hyperparams is a 1D-tensor
        hyperparam_values = np.asarray(hyperparam_values).ravel()

        model_hyperparams = {hname: _ceil(hvalue) for hname, hvalue in zip(hyperparams, hyperparam_values)}
        model = model_class(image_sz, n_classes, **model_hyperparams)
        model = model.to(_device)
        loss_fn = F.nll_loss

        # The last hyperparam is the learning rate. We could use momentum, whatever as well
        nn_optimizer = torch_opt.SGD(model.parameters(), lr=hyperparam_values[-1], momentum=hyperparam_values[-2])

        filename = '{}_{}'.format(model_prefix, callno)
        cbs = [Checkpoint('val_accuracy', min_delta=1e-3, filename=filename, save_best=True, increasing=True)]

        # Training
        history = trainer.train_on_loader(model, trn_gen, val_gen, loss_fn, nn_optimizer,
                                          n_epochs=n_epochs, batch_first=True, device=_device,
                                          callbacks=cbs)
        
        # Getting the best model during training to evaluate
        best_model = torch.load(filename + '.pth').to(_device)
        eval_metrics = trainer.evaluate_on_loader(best_model, val_gen, loss_fn, batch_first=True,
                                                  device=_device, verbosity=0)
        
        # Opytimizer minimizes functions
        fitness = 1 - eval_metrics['accuracy']
        scoreboard[callno] = fitness
        callno += 1        
        return fitness
    return target_fn

In [4]:
from opytimizer.optimizers.fa import FA
from models import utils

BATCH_SIZE = 128
N_AGENTS = 2
N_ITERATIONS = 2

train_loader, val_loader, tst_loader = datasets.mnist_laoders(BATCH_SIZE, trn_split_sz=0.8)

target_fn = make_target_fn('./trained/firefly',
                           device,
                           ConvNet,
                           train_loader,
                           val_loader,
                           n_epochs=3,
                           image_sz=28,
                           n_classes=10,
                           hyperparams=['filters_1'])

# filter_1, lr, momentum
lower_bound = [1, 1e-3, 0]
upper_bound = [20, 1e-2, 1]

n_variables = len(lower_bound)
meta_hyperparams = dict(alpha=0.5, beta=0.2, gamma=1.0)

start = time.time()
history = utils.optimize(FA,
                         target=target_fn,
                         n_agents=N_AGENTS,
                         n_variables=n_variables,
                         n_iterations=N_ITERATIONS,
                         lb=lower_bound,
                         ub=upper_bound,
                         hyperparams=meta_hyperparams)
end = time.time() - start
print(end)

2019-08-02 13:36:15,036 - opytimizer.spaces.search — INFO — Overriding class: Space -> SearchSpace.
2019-08-02 13:36:15,039 - opytimizer.core.space — DEBUG — Running private method: build().
2019-08-02 13:36:15,040 - opytimizer.core.space — DEBUG — Running private method: check_bound_size().
2019-08-02 13:36:15,040 - opytimizer.core.space — DEBUG — Bound checked.
2019-08-02 13:36:15,041 - opytimizer.core.space — DEBUG — Running private method: check_bound_size().
2019-08-02 13:36:15,041 - opytimizer.core.space — DEBUG — Bound checked.
2019-08-02 13:36:15,042 - opytimizer.core.space — DEBUG — Running private method: create_agents().
2019-08-02 13:36:15,043 - opytimizer.core.space — DEBUG — Agents: 2 | Size: (3, 1) | Iterations: 2 | Lower Bound: [1, 0.001, 0] | Upper Bound: [20, 0.01, 1] | Built: True.
2019-08-02 13:36:15,044 - opytimizer.spaces.search — DEBUG — Running private method: initialize_agents().
2019-08-02 13:36:15,045 - opytimizer.spaces.search — DEBUG — Agents initialized.
2


2019-08-02 13:38:03,011 - opytimizer.optimizers.fa — INFO — Iteration 1/2



2019-08-02 13:39:53,556 - opytimizer.optimizers.fa — INFO — Fitness: 0.021583333333333288
2019-08-02 13:39:53,557 - opytimizer.optimizers.fa — INFO — Position: [[1.18033328e+01]
 [4.54766297e-03]
 [1.71350458e-01]]
2019-08-02 13:39:53,558 - opytimizer.optimizers.fa — INFO — Iteration 2/2



2019-08-02 13:41:46,988 - opytimizer.optimizers.fa — INFO — Fitness: 0.021583333333333288
2019-08-02 13:41:46,989 - opytimizer.optimizers.fa — INFO — Position: [[1.18033328e+01]
 [4.54766297e-03]
 [1.71350458e-01]]
2019-08-02 13:41:46,990 - opytimizer.opytimizer — INFO — Optimization task ended.
2019-08-02 13:41:46,991 - opytimizer.opytimizer — INFO — It took 331.9334080219269 seconds.
331.9567131996155


In [12]:
top_indices, top_fitness = utils.get_top_models(scoreboard, 2)
for ti, tf in zip(top_indices, top_fitness):
    print(f'{ti:<5} {tf:5}')


best_models = utils.load_models('./trained', 'firefly', top_indices)
for idx, model in zip(top_indices, best_models):
    utils.predict_persist(model, val_loader, device, f'predictions/firefly_{idx}.txt')

utils.store_labels(val_loader, './predictions/labels.txt')

0     0.021583333333333288
4     0.02300000000000002
